In [1]:
# 导入相关模块
import pandas as pd
from tensorflow.keras import metrics
import tensorflow.keras.backend as K
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import numpy as np
import os
from skimage import io
from tqdm import tqdm

In [10]:
# # 设置GPU
# gpus = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
# for gpu in gpus:
#     tf.config.experimental.set_memory_growth(gpu, True)
# print(len(gpus))
# logical_gpus = tf.config.experimental.list_logical_devices('GPU')
# print(len(logical_gpus))

In [11]:
#读取数据
jan_train = pd.read_csv('./TrainingNew/trainingJunNum1.csv')

In [12]:
#数据归一化
jan_train_x = pd.concat([jan_train.iloc[:,1:8]/jan_train.iloc[:,1:8].max(),
                         jan_train.iloc[:,9:12]/jan_train.iloc[:,9:12].max()],axis=1,join='inner').values
jan_train_y = jan_train.landcover.values

In [13]:
#划分数据集
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(jan_train_x, jan_train_y, test_size=0.2, random_state=3)
print(len(X_train), len(X_test))

960 240


In [14]:
#设置训练模型相关内容，详情见 https://keras.io/zh/callbacks/
logdir = os.path.join("model")
if not os.path.exists(logdir):
    os.mkdir(logdir)
output_model_file = os.path.join(logdir,"model.h5")

callbacks = [
    keras.callbacks.TensorBoard(logdir),
    keras.callbacks.ModelCheckpoint(output_model_file,monitor='val_accuracy',save_best_only = True,save_weights_only = False),
    keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=100, min_delta=1e-7),
]

In [15]:
#定义dnn网络结构
model = Sequential()
model.add(Dense(32, input_shape=(10,)))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Dense(6))
model.add(Activation('softmax'))
#设置loss函数、优化器等
model.compile(loss="sparse_categorical_crossentropy",optimizer = tf.keras.optimizers.Adam(),metrics = ["accuracy"])
#训练模型
model.fit(X_train, Y_train, epochs=100, batch_size=64,shuffle=True,
          validation_data=(X_test,Y_test),callbacks=callbacks,verbose=1)

Train on 960 samples, validate on 240 samples
Epoch 1/100
960/960 [==============================] - 1s 625us/sample - loss: 1.7682 - accuracy: 0.2167 - val_loss: 1.7169 - val_accuracy: 0.3417
Epoch 2/100
960/960 [==============================] - 0s 74us/sample - loss: 1.6910 - accuracy: 0.3990 - val_loss: 1.6494 - val_accuracy: 0.4958
Epoch 3/100
960/960 [==============================] - 0s 51us/sample - loss: 1.6109 - accuracy: 0.4646 - val_loss: 1.5693 - val_accuracy: 0.4958
Epoch 4/100
960/960 [==============================] - 0s 65us/sample - loss: 1.5241 - accuracy: 0.4958 - val_loss: 1.4764 - val_accuracy: 0.5125
Epoch 5/100
960/960 [==============================] - 0s 68us/sample - loss: 1.4314 - accuracy: 0.5271 - val_loss: 1.3661 - val_accuracy: 0.5583
Epoch 6/100
960/960 [==============================] - ETA: 0s - loss: 1.3680 - accuracy: 0.53 - 0s 68us/sample - loss: 1.3124 - accuracy: 0.5646 - val_loss: 1.2499 - val_accuracy: 0.5833
Epoch 7/100
960/960 [==============

960/960 [==============================] - 0s 45us/sample - loss: 0.1942 - accuracy: 0.9250 - val_loss: 0.1424 - val_accuracy: 0.9542
Epoch 57/100
960/960 [==============================] - 0s 70us/sample - loss: 0.1939 - accuracy: 0.9333 - val_loss: 0.1325 - val_accuracy: 0.9750
Epoch 58/100
960/960 [==============================] - 0s 47us/sample - loss: 0.1948 - accuracy: 0.9344 - val_loss: 0.1353 - val_accuracy: 0.9708
Epoch 59/100
960/960 [==============================] - 0s 40us/sample - loss: 0.1934 - accuracy: 0.9219 - val_loss: 0.1286 - val_accuracy: 0.9708
Epoch 60/100
960/960 [==============================] - 0s 45us/sample - loss: 0.1895 - accuracy: 0.9292 - val_loss: 0.1379 - val_accuracy: 0.9500
Epoch 61/100
960/960 [==============================] - 0s 45us/sample - loss: 0.1942 - accuracy: 0.9260 - val_loss: 0.1263 - val_accuracy: 0.9750
Epoch 62/100
960/960 [==============================] - 0s 43us/sample - loss: 0.1835 - accuracy: 0.9312 - val_loss: 0.1285 - val_a

In [16]:
#加载训练好的模型
model = keras.models.load_model(output_model_file)

In [17]:
#读取测试图片
jan_img = np.reshape(io.imread('./6June/Junemask.tif'),(-1,13))
jan_img

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [18]:
h,w,c = io.imread('./6June/Junemask.tif').shape

In [19]:
jan_img_b2_b8 = jan_img[:,1:8]

In [20]:
#测试图片数据预处理
ndd = jan_img[:,3]/jan_img[:,7]
ndsi = (jan_img[:,2]-jan_img[:,11])/(jan_img[:,2]+jan_img[:,11])
ndvi = (jan_img[:,7]-jan_img[:,3])/(jan_img[:,7]+jan_img[:,3])
jan_test_data = np.c_[jan_img_b2_b8,ndd,ndsi,ndvi]/np.concatenate((jan_train.iloc[:,1:8].max().values,jan_train.iloc[:,9:12].max().values))

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


In [ ]:
#预测结果
pred = []
invalid_data = []
valid_data = []
for i in tqdm(range(len(jan_img))):
    if jan_test_data[i][0] == 0 or jan_test_data[i][0] == -1:
        pred.append(-1)
    else:
        pred.append(model.predict(jan_test_data[i][np.newaxis,::]).argmax())

  0%|          | 34251/9929820 [16:32<99:42:45, 27.57it/s]  

In [ ]:
#设置不同label的颜色
colour = []
for r in [0,255]:
    for g in [0,255]:
        for b in [0,255]:
            colour.append([r,g,b])

colour

In [ ]:
# 将预测的 0-5 标签，转化为相应的颜色，并reshape
pred_map = []
for j in pred:
    pred_map.append(colour[j+1])
    
pred_map = np.array(pred_map).reshape((h,w,3)).astype(np.uint8)

In [8]:
# 画图
import matplotlib.pyplot as plt
%matplotlib inline
plt.imshow(pred_map)
plt.colorbar()
plt.show()

NameError: name 'pred_map' is not defined

In [9]:
#保存图片
from PIL import Image
Image.fromarray(pred_map).save('jun_img.jpg')

NameError: name 'pred_map' is not defined